# Data Munging

In [1]:
import pandas as pd

In [77]:
raw_train_data = pd.read_csv("sales_train.csv")
category_data = pd.read_csv("items.csv").loc[:,["item_id", "item_category_id"]]
category_data_dict = {x[0]:x[1] for x in category_data.itertuples(index = False)}
sample_train = raw_train_data.sample(frac=0.001, replace = True).loc[:, ["date_block_num", "shop_id", "item_id", "item_price", "item_cnt_day"]]

# One hot encoding//pending

# Abalation//pending

In [78]:
# def getSubSampleTrainTestSet(data_set,):
sample_item_set = set(sample_train.item_id)
len(sample_item_set)
def isInTraining(item_id):
    if item_id in sample_item_set:
        return True
    return False
def getCategoryIdForItem(item_id):
#     return category_data.loc[category_data["item_id"] == item_id , "item_id"]
    return category_data_dict[item_id]



In [76]:
print(getCategoryIdForItem(1723))


55
